# Assignment 4: Nelson Herrera

In [1]:
import pygame
import moderngl
import numpy as np
import time
import glm
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
# Set the version of OpenGL. Initialize OpenGL context using Pygame
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 1)
pygame.display.set_mode((800,600), pygame.OPENGL | pygame.DOUBLEBUF)

# Get OpenGL context associated with the window
gl = moderngl.get_context()
gl.enable(gl.DEPTH_TEST)

2024-09-23 22:37:52.925 Python[69606:1751598] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


## Import the object files and upload them to the GPU

In [3]:

vertexData, bounds = getObjectData("Teapot.obj", normal=True)

print("vertexData:", vertexData)
print("\n")
print("bounds: ", bounds)
print("bound center:", bounds.center)

# # Upload Vertex data to GPU
vertexBuffer = gl.buffer(vertexData)

Normal exists
vertexData: [ 0.605903  0.005903 -0.       ...  0.801569 -0.558184 -0.214148]


bounds:  boundingBox:[array([-3.,  0., -2.]), array([3.42963, 3.15   , 2.     ])], enter: [0.214815 1.575    0.      ], Radius: 4.100690244852078.
bound center: [0.214815 1.575    0.      ]


In [4]:
translateMatrix = glm.translate(glm.mat4(1), -glm.vec3(bounds.center))
scaleMatrix = glm.scale(glm.mat4(1), glm.vec3(1.0 / bounds.radius))  # Normalize based on the object's size
modelMatrix = scaleMatrix * translateMatrix

In [5]:
# Vertex Shader
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec4 in_position;
layout (location = 1) in vec3 in_normal;
uniform mat4 modelMatrix;

out vec3 f_normal;

void main() {
    f_normal = in_normal;
    gl_Position = modelMatrix*in_position;
}
'''

In [6]:
# Fragment Shader
fragment_shader_code = '''
# version 330 core 

in vec3 f_normal;

layout (location = 0) out vec4 out_color;

void main() {
    out_color = vec4(0.5*(normalize(f_normal) + 1.0), 1.0);
}
'''

In [7]:
# Store them in the main shader program
shaderProgram = gl.program(
    vertex_shader = vertex_shader_code,
    fragment_shader = fragment_shader_code
)

shaderProgram['modelMatrix'].write(modelMatrix)


In [8]:
renderable = gl.vertex_array(shaderProgram, [(vertexBuffer, '3f 3f', 'in_position', 'in_normal')])


In [9]:
pygame.display.set_caption("Assignment 4: Nelson Herrera")

running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
    gl.clear(0.2,0.2,0.2)
    renderable.render()
    pygame.display.flip()

pygame.quit()  # quit after the loop ends
quit() # Needed this to quit the process on my Mac without jupyter notebook freezing